In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.callbacks import EarlyStopping

In [28]:
#Loading dataset
data = pd.read_csv('fer2013.csv')
pixels = data['pixels'].tolist()

In [29]:
# Pre-processing: each pixels array is processed into a numpy array and added to the set
X = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(48, 48)
    face = np.expand_dims(face, axis=-1)
    X.append(face)

X = np.asarray(X)
X = X.astype('float32')
X /= 255.0  # Normlization [0,1]

# Getting the predicted labeles from the set
y = pd.get_dummies(data['emotion']).values  # One-hot encoding
y_labels = np.argmax(y, axis=1)  # Convertire da one-hot encoding a etichette

# Divisione in train e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# StratifiedShuffleSplit per mantenere la distribuzione delle classi
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(X, y_labels):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


#Eraly stopping callback to pre
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [31]:
#CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\acaia\anaconda3\envs\first_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
#Fitting the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - accuracy: 0.2403 - loss: 2.3918 - val_accuracy: 0.3367 - val_loss: 1.6869
Epoch 2/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 40s 89ms/step - accuracy: 0.3549 - loss: 1.6809 - val_accuracy: 0.4465 - val_loss: 1.5205
Epoch 3/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 39s 87ms/step - accuracy: 0.4220 - loss: 1.5499 - val_accuracy: 0.4451 - val_loss: 1.4803
Epoch 4/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 40s 89ms/step - accuracy: 0.4535 - loss: 1.4868 - val_accuracy: 0.5007 - val_loss: 1.4158
Epoch 5/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 42s 93ms/step - accuracy: 0.4810 - loss: 1.4291 - val_accuracy: 0.5043 - val_loss: 1.3807
Epoch 6/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 40s 90ms/step - accuracy: 0.4938 - loss: 1.3926 - val_accuracy: 0.5146 - val_loss: 1.3402
Epoch 7/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 40s 88ms/step - accuracy: 0.5017 - loss: 1.3742 - val_accuracy: 0.5298 - val_loss: 1.3226
Epoch 8/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.5225 - loss: 1

In [33]:
#Testing
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

#Metrics calculation
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step
Accuracy: 0.5739760378935637
Precision: 0.5557117385583445
Recall: 0.5739760378935637
F1 Score: 0.5621278440023246


c:\Users\acaia\anaconda3\envs\first_env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
model.save('cnn_FER2013.h5')